## Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [2]:
purchase_data.head()


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
purchase_data.shape

(780, 7)

## Player Count
* Display the total number of players

In [4]:
pd.DataFrame({"Total Players": [len(purchase_data["SN"].unique())]})

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [5]:
items = len(purchase_data["Item Name"].unique())
average_price = purchase_data["Price"].mean()
purchases = len(purchase_data["Purchase ID"])
revenue = purchase_data["Price"].sum()

In [6]:
purchasing_analysis = pd.DataFrame({"Number of Unique Items":[items],
             "Average Price":[average_price],
             "Number of Purchases":[purchases],
             "Total Revenue":[revenue]})

purchasing_analysis["Number of Unique Items"] = purchasing_analysis["Number of Unique Items"].map("{:.0f}".format)
purchasing_analysis["Average Price"] = purchasing_analysis["Average Price"].map("${:.2f}".format)
purchasing_analysis["Number of Purchases"] = purchasing_analysis["Number of Purchases"].map("{:.0f}".format)
purchasing_analysis["Total Revenue"] = purchasing_analysis["Total Revenue"].map("${:.2f}".format)
purchasing_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


## Gender Demographics
* Percentage and Count of Male Players
* Percentage and COunt of Female Players
* Percentage and Counf of Other / Non-Disclosed


In [7]:
gender_df = purchase_data.drop_duplicates(subset=["SN"])
gender_df = gender_df["Gender"].value_counts().to_frame() 
gender_df = gender_df.rename(columns={"Gender":"Total Count"})
gender_df["Percentage of Players"] = (gender_df["Total Count"] / gender_df["Total Count"].sum())
gender_df = gender_df.style.format({"Percentage of Players":'{:.2%}'})
gender_df

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [8]:
purchasing_df = purchase_data["Gender"].value_counts().to_frame()
purchasing_df = purchasing_df.rename(columns={"Gender":"Purchase Count"})
purchasing_df = purchasing_df.sort_index(ascending=True)
purchase_mean = purchase_data.groupby(["Gender"]).mean()
purchase_sum = purchase_data.groupby(["Gender"]).sum()
avg_per_person = purchase_data.groupby(["Gender","SN"]).sum()
avg_per_person = avg_per_person.reset_index()
avg_per_person = avg_per_person.groupby(["Gender"]).mean()
purchasing_df["Average Purchase Price"] = purchase_mean["Price"]
purchasing_df["Total Purchase Value"] = purchase_sum["Price"]
purchasing_df["Avg Total Purchase per Person"] = avg_per_person["Price"]
purchasing_df = purchasing_df.style.format({"Average Purchase Price":"${:.2f}",
                                           "Total Purchase Value":"${:.2f}",
                                           "Avg Total Purchase per Person":"${:.2f}"})
purchasing_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table

In [9]:
bins = [0,9,14,19,24,29,34,39,max(purchase_data["Age"])]
age_groups = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

In [10]:
age_df = purchase_data.drop_duplicates(subset=["SN"])
age_df["Ranges"] = pd.cut(age_df["Age"],bins,labels=age_groups, include_lowest=True)
age_df = age_df.groupby(["Ranges"]).count()
age_df["Percentage of Players"] = age_df["Age"] / age_df["Age"].sum()
age_df_filter = age_df.filter(["Age","Percentage of Players"])
age_df_filter.rename(columns={"Age":"Total Count"})
age_df.reset_index()
age_df_filter = age_df_filter.style.format({"Percentage of Players":"{:.2%}"})
age_df_filter

<ipython-input-10-56c8305e7f13>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age_df["Ranges"] = pd.cut(age_df["Age"],bins,labels=age_groups, include_lowest=True)


,Age,Percentage of Players
Ranges,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [11]:
age_analysis_df = purchase_data
age_analysis_df["Ranges"] = pd.cut(age_analysis_df["Age"],bins=bins,labels=age_groups,include_lowest=True)
age_analysis_df = age_analysis_df.groupby(["Ranges"])
age_analysis_clean = pd.DataFrame(age_analysis_df["Purchase ID"].count())
age_analysis_clean.rename(columns={"Purchase ID":"Purchase Count"})
age_analysis_clean["Average Purchase Price"] = age_analysis_df["Price"].mean() /100
age_analysis_clean["Total Purchase Value"] = age_analysis_df["Price"].sum()
age_analysis_clean["Avg Total Purchase per Person"] = (age_analysis_clean["Total Purchase Value"] / age_df["SN"]) / 100
age_analysis_clean = age_analysis_clean.style.format({"Average Purchase Price":"{:.2%}",
                                                     "Total Purchase Value":"${:.2f}",
                                                     "Avg Total Purchase per Person":"{:.2%}"})
age_analysis_clean

,Purchase ID,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Ranges,,,,
<10,23,3.35%,$77.13,4.54%
10-14,28,2.96%,$82.78,3.76%
15-19,136,3.04%,$412.89,3.86%
20-24,365,3.05%,$1114.06,4.32%
25-29,101,2.90%,$293.00,3.81%
30-34,73,2.93%,$214.00,4.12%
35-39,41,3.60%,$147.67,4.76%
40+,13,2.94%,$38.24,3.19%


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame


In [12]:
spenders_df = purchase_data[["SN","Item ID","Price"]]
spenders_df = spenders_df.groupby(["SN"])
spenders_clean = pd.DataFrame(spenders_df["Item ID"].count())
spenders_clean = spenders_clean.rename(columns={"Item ID":"Purchase Count"})
spenders_clean["Average Purchase Price"] = spenders_df["Price"].mean()
spenders_clean["Total Purchase Value"] = spenders_df["Price"].sum()
spenders_clean = spenders_clean.nlargest(5,"Total Purchase Value")
spenders_clean = spenders_clean.style.format({"Average Purchase Price":"${:.2f}",
                                             "Total Purchase Value":"${:.2f}"})
spenders_clean

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

In [13]:
popular_df = purchase_data[["Item ID","Item Name","Price"]]
popular_df = popular_df.groupby(["Item ID","Item Name"])
popular_clean = pd.DataFrame(popular_df["Price"].count())
popular_clean = popular_clean.rename(columns={"Price":"Purchase Count"})
popular_clean["Item Price"] = popular_df["Price"].sum() / popular_df["Price"].count()
popular_clean["Total Purchase Value"] = popular_df["Price"].sum()
popular_clean = popular_clean.sort_values(by="Purchase Count", ascending=False)
popular_summary = popular_clean.head(5).style.format({"Item Price":"${:.2f}",
                          "Total Purchase Value":"${:.2f}"})
popular_summary

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [14]:
profit_items = popular_clean.sort_values(by="Total Purchase Value", ascending=False)
profit_items = profit_items.head(5).style.format({"Item Price":"${:.2f}",
                          "Total Purchase Value":"${:.2f}"})
profit_items

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
